In [1]:
import pandas as pd

# Weather

In [2]:
# Data from NYC - Central Park https://www.weather.gov/wrh/Climate?wfo=okx
df_weather = pd.read_csv('../data/external/weather_data.csv', delimiter=';')

In [3]:
df_weather = df_weather.sort_values(by=['Type', 'Month', 'Day'], ascending=[True, True, True])
df_weather.Type.value_counts()

Type
Avg Temperature Normal                 372
Daily Precipitation Normal (inches)    372
Max Temperature Normal                 372
Min Temperature Normal                 372
Name: count, dtype: int64

In [4]:
# This works because dataframe is ordered
df_weather_unify = df_weather[df_weather['Type'] == 'Daily Precipitation Normal (inches)'][['Day', 'Month', 'Value']].rename(columns={'Value' : 'daily_preciptation_normal_inches'}).reset_index(drop=True)
        
df_weather_unify['max_temperature_normal_f'] = df_weather[df_weather['Type'] == 'Max Temperature Normal']['Value'].values

df_weather_unify['min_temperature_normal_f'] = df_weather[df_weather['Type'] == 'Min Temperature Normal']['Value'].values

df_weather_unify['avg_temperature_normal_f'] = df_weather[df_weather['Type'] == 'Avg Temperature Normal']['Value'].values

df_weather_unify['Year'] = 2016

In [5]:
df_weather_unify = df_weather_unify.replace('-', '0')

In [11]:
df_weather_unify.to_csv('../data/external/weather_unify.csv')